In [1]:
import numpy as np
import pandas as pd

from math import sin, cos, atan2, sqrt, radians

from scrapy.selector import Selector
import requests
from bs4 import BeautifulSoup
import os
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from datetime import datetime
from time import sleep, strptime

%matplotlib inline

chromedriver = './chromedriver.exe'

# streets_csv = './datasets/(to scrape) street_names.csv'
street_lon_lat_csv = './datasets/street_lon_lat.csv'
stations_lon_lat_csv = './datasets/stations_lon_lat.csv'

# streets = pd.read_csv(streets_csv, index_col=0)
streets_lon_lat = pd.read_csv(street_lon_lat_csv, index_col=0)
stations_lon_lat = pd.read_csv(stations_lon_lat_csv, index_col=0)

In [2]:
streets_lon_lat[streets_lon_lat.index == 'HILLVIEW AVE BLK 6']

,Latitude,Longitude


In [77]:
stations_lon_lat.head()

,Opening Date,Latitude,Longitude
Stations,,,
Jurong East MRT station,1990-03-10,1.333131,103.742091
Bukit Batok MRT station,1990-03-10,1.348506,103.749222
Bukit Gombak MRT station,1990-03-10,1.359037,103.751828
Choa Chu Kang MRT/LRT station,1990-03-10,1.385385,103.744337
Yew Tee MRT station,1996-02-10,1.397329,103.747502


## Scraping Function

In [62]:
def scraping(scrape_list, item_dict, backup_name):

    main_url = 'https://www.google.com.sg/maps'

    driver = webdriver.Chrome(chromedriver)

    driver.get(main_url)

    mapsearch = driver.find_element_by_id('searchboxinput')

    consec_fail = 0

    for i, add in enumerate(scrape_list):

        tries = 0
        mapsearch.clear()
        mapsearch.send_keys(add)
        mapsearch.send_keys(Keys.RETURN)

        if i == 0:
            sleep(5)
        else:
            while item_dict[prev_add] == driver.current_url[driver.current_url.find('!3d')+3 :].split('!4d') or driver.current_url.find('!3d') == -1:
                sleep(1)
                tries += 1
                if tries % 10 == 0:
                    mapsearch.clear()
                    mapsearch.send_keys(add)
                    mapsearch.send_keys(Keys.RETURN)
                    sleep(5)
                elif tries > 20:
                    break

        if tries <= 20:
            item_dict[add] = driver.current_url[driver.current_url.find('!3d')+3 :].split('!4d')
            consec_fail = 0
        else:
            item_dict[add] = ['attempt', 'rerun']
            consec_fail += 1

        if i != 0 and i % 100 == 0:
            pd.DataFrame(item_dict, index=['Latitude', 'Longitude']).T.to_csv('./backup/{} - {}.csv'.format(backup_name, i))

        prev_add = add

        if consec_fail > 2:
            driver.close()
            sleep(2)
            driver = webdriver.Chrome(chromedriver)
            driver.get(main_url)
            mapsearch = driver.find_element_by_id('searchboxinput')
            consec_fail = 0
            sleep(2)

    driver.close()
    
    return item_dict

## Scrape Flat Locations

In [ ]:
# long_lat_dict = {}
# long_lat_dict = scraping(streets['combined'], long_lat_dict, backup_name='block_names')
# scraped = pd.DataFrame(long_lat_dict, index=['Latitude', 'Longitude']).T
# scraped['Latitude'] = scraped['Latitude'].apply(lambda x: x.strip())
# scraped['Longitude'] = scraped['Longitude'].apply(lambda x: x.strip())
# scraped.to_csv('./FinalScraped.csv')

In [ ]:
# # Identify if there are any wrong info scraped

# scraped['rescrape'] = scraped['Latitude'].apply(lambda x: 0 if x.find('/') == -1 else 1)
# scraped[scraped['rescrape'] == 1]

In [ ]:
# # Will throw an error if wrong info scraped too

# scraped['Longitude'] = scraped['Longitude'].astype(float)
# scraped['Latitude'] = scraped['Latitude'].astype(float)

## Scrape Train Stations

In [ ]:
# # Scrape all train stations available

# train_station_link = 'https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations'

# driver = webdriver.Chrome(chromedriver)
# driver.get(train_station_link)

# HTML = driver.page_source
# # HTML = BeautifulSoup(HTML, 'lxml')

# driver.close()


# train_stations = {}
# train_stations['Stations'] = []

# for row in Selector(text=HTML).xpath("//*[@class='wikitable']/*/tr/td[2]").extract():
#     train_stations['Stations'] += [row[row.find('title=')+7 : row.find('"', row.find('title=')+7)]]

# stations = pd.DataFrame(train_stations)
# stations.drop(index=stations[stations['Stations'] == 'lspan='].index, inplace=True)

# train_stations['Opening Date'] = Selector(text=HTML).xpath("//*[@class='wikitable']/*/tr/td[5]/text()").extract()
    
# stations.insert(loc=1, column='Opening Date', value=train_stations['Opening Date'])
# stations.drop(index=stations[stations['Opening Date'] == 'TBA'].index, inplace=True)

# stations.drop_duplicates(subset=['Stations'], inplace=True)

# stations['Opening Date'] = pd.to_datetime(stations['Opening Date'])
# stations['To_Drop'] = stations['Opening Date'].apply(lambda x: 1 if x > datetime.strptime('2018-10-01', '%Y-%m-%d') else 0)

# stations = stations[stations['To_Drop'] == 0][['Stations', 'Opening Date']]

# stations.reset_index(drop=True, inplace=True)

In [ ]:
# stations_lon_lat_dict = {}

# stations_lon_lat_dict = scraping(stations['Stations'], stations_lon_lat_dict, backup_name='Stations_loc')

# stations_lon_lat_df = pd.DataFrame(stations_lon_lat_dict, index=['Latitude', 'Longitude']).T

In [ ]:
# stations_lon_lat_df['Latitude'] = stations_lon_lat_df['Latitude'].astype(float)
# stations_lon_lat_df['Longitude'] = stations_lon_lat_df['Longitude'].astype(float)

In [ ]:
# stations.set_index(keys='Stations', inplace=True)
# stations = stations.merge(stations_lon_lat_df[['Latitude', 'Longitude']], left_index=True, right_index=True)
# stations.to_csv('./datasets/stations_lon_lat.csv')

## Scrape School Location

## Distance Measurement
- a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
- c = 2 ⋅ atan2( √a, √(1−a) )
- d = R ⋅ c

In [5]:
def distance(lon1, lat1, lon2, lat2):
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    
    a = (sin(abs(lat1 - lat2) / 2))**2 + cos(lat1) * cos(lat2) * sin(abs(lon1 - lon2) / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return 6371000 * c
    

## Calculate Distance to Train Stations and find top 5 stations
- Reason for finding top 5 is because I'm trying to find number of stations within 500m radius
- The smallest number of stations possible to do so is 5 stations

In [80]:
nearest_5_stations = {}

for i in streets_lon_lat.index:
    
    flat_station_distance = []
    
    for j in stations_lon_lat.index:
        
        flat_station_distance.append(distance(lon1=streets_lon_lat.loc[i, 'Longitude'], lat1=streets_lon_lat.loc[i, 'Latitude'], lon2=stations_lon_lat.loc[j, 'Longitude'], lat2=stations_lon_lat.loc[j, 'Latitude']))
        
    nearest_5_stations[i] = sorted(list(zip(stations_lon_lat.index, flat_station_distance)), key=lambda x: x[1], reverse=False)[:5]

In [95]:
flat_station_distance = pd.DataFrame(nearest_5_stations, index=['Station 1', 'Station 2', 'Station 3', 'Station 4', 'Station 5']).T
flat_station_distance['S1'] = flat_station_distance['Station 1'].apply(lambda x: x[0])
flat_station_distance['S2'] = flat_station_distance['Station 2'].apply(lambda x: x[0])
flat_station_distance['S3'] = flat_station_distance['Station 3'].apply(lambda x: x[0])
flat_station_distance['S4'] = flat_station_distance['Station 4'].apply(lambda x: x[0])
flat_station_distance['S5'] = flat_station_distance['Station 5'].apply(lambda x: x[0])
flat_station_distance['Distance to S1'] = flat_station_distance['Station 1'].apply(lambda x: x[1])
flat_station_distance['Distance to S2'] = flat_station_distance['Station 2'].apply(lambda x: x[1])
flat_station_distance['Distance to S3'] = flat_station_distance['Station 3'].apply(lambda x: x[1])
flat_station_distance['Distance to S4'] = flat_station_distance['Station 4'].apply(lambda x: x[1])
flat_station_distance['Distance to S5'] = flat_station_distance['Station 5'].apply(lambda x: x[1])
flat_station_distance['S1 Opening Date'] = float(np.nan)
flat_station_distance['S2 Opening Date'] = float(np.nan)
flat_station_distance['S3 Opening Date'] = float(np.nan)
flat_station_distance['S4 Opening Date'] = float(np.nan)
flat_station_distance['S5 Opening Date'] = float(np.nan)
flat_station_distance.drop(columns=['Station 1', 'Station 2', 'Station 3', 'Station 4', 'Station 5'], inplace=True)

for station in flat_station_distance['S1']:
    flat_station_distance['S1 Opening Date'] = stations_lon_lat.loc[station, 'Opening Date']
    
for station in flat_station_distance['S2']:
    flat_station_distance['S2 Opening Date'] = stations_lon_lat.loc[station, 'Opening Date']
    
for station in flat_station_distance['S3']:
    flat_station_distance['S3 Opening Date'] = stations_lon_lat.loc[station, 'Opening Date']
    
for station in flat_station_distance['S4']:
    flat_station_distance['S4 Opening Date'] = stations_lon_lat.loc[station, 'Opening Date']
    
for station in flat_station_distance['S5']:
    flat_station_distance['S5 Opening Date'] = stations_lon_lat.loc[station, 'Opening Date']

In [83]:
flat_station_distance.to_csv('./datasets/flat_station_distance.csv')

In [106]:
cutoff_dist = 500

for col in ['Distance to S2', 'Distance to S3', 'Distance to S4', 'Distance to S5']:
    flat_station_distance[col] = flat_station_distance[col].apply(lambda x: 0 if (x > cutoff_dist or x == 0) else 1)

flat_station_distance['Number of Stations within 500m'] = flat_station_distance['Distance to S1'].apply(lambda x: 0 if (x > cutoff_dist or x == 0) else 1) + flat_station_distance['Distance to S2'] + flat_station_distance['Distance to S3'] + flat_station_distance['Distance to S4'] + flat_station_distance['Distance to S5']

In [107]:
flat_station_distance['Number of Stations within 500m'].value_counts()

0    6016
1    2761
2      91
3      13
4       4
5       1
Name: Number of Stations within 500m, dtype: int64

In [108]:
flat_station_distance

,S1,S2,S3,S4,S5,Distance to S1,Distance to S2,Distance to S3,Distance to S4,Distance to S5,S1 Opening Date,S2 Opening Date,S3 Opening Date,S4 Opening Date,S5 Opening Date,Number of Stations within 500m
ANG MO KIO AVE 1 BLK 309,Ang Mo Kio MRT station,Bishan MRT station,Marymount MRT station,Yio Chu Kang MRT station,Braddell MRT station,874.344141,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 1 BLK 216,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Marymount MRT station,Bishan MRT station,Braddell MRT station,1000.104741,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 3 BLK 211,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Bishan MRT station,Marymount MRT station,Braddell MRT station,892.208593,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 3 BLK 202,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Bishan MRT station,Marymount MRT station,Lorong Chuan MRT station,580.749042,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 3 BLK 235,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Marymount MRT station,Bishan MRT station,Braddell MRT station,1525.154985,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 3 BLK 232,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Marymount MRT station,Bishan MRT station,Braddell MRT station,1385.413939,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 1 BLK 308,Ang Mo Kio MRT station,Bishan MRT station,Marymount MRT station,Yio Chu Kang MRT station,Braddell MRT station,858.450328,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 1 BLK 220,Ang Mo Kio MRT station,Marymount MRT station,Yio Chu Kang MRT station,Bishan MRT station,Braddell MRT station,1127.767705,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 1 BLK 219,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Marymount MRT station,Bishan MRT station,Braddell MRT station,1101.726850,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0
ANG MO KIO AVE 3 BLK 247,Ang Mo Kio MRT station,Yio Chu Kang MRT station,Marymount MRT station,Bishan MRT station,Braddell MRT station,1729.803748,0,0,0,0,1988-12-20,1988-12-20,1996-02-10,1996-02-10,1987-11-07,0


## Scrape Primary School

In [169]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://en.wikipedia.org/wiki/List_of_primary_schools_in_Singapore')

HTML = driver.page_source

driver.close()

In [170]:
school_info = Selector(text=HTML).xpath('//*[@class="wikitable sortable"]/tbody/tr/td/text()').extract()
schools = [col for col in [col for col in school_info if '\n' not in col] if ('Used' not in col and col not in ['Government', 'Mixed', 'Government-aided', 'Government-aided, ', 'Affiliated to ', 'Boys', 'Girls'])]
schools = [col for col in schools if (('School' in col or 'CHIJ' in col or 'school' in col or 'Joseph' in col) and 'Merg' not in col)]

In [171]:
for i, school in enumerate(schools):
    schools[i] = school.strip().lower().replace(',','')

schools = np.unique(schools)

In [174]:
# school_lon_lat = {}
scraping(school_lon_lat_df[(school_lon_lat_df['Latitude'] == 'attempt') | (school_lon_lat_df['Flag'] == 1)].index, school_lon_lat, 'Primary School Loc')

{'admiralty primary school': ['1.4430452', '103.8003087'],
 'ahmad ibrahim primary school': ['1.4337349', '103.8327825'],
 'alexandra primary school': ['1.2912857', '103.8235733'],
 'anchor green primary school': ['1.3909254', '103.8874074'],
 'anderson primary school': ['1.384692', '103.841225'],
 'ang mo kio primary school': ['1.3693469', '103.839815'],
 'anglo-chinese school (junior)': ['1.308957', '103.841503'],
 'anglo-chinese school (primary)': ['1.3186818', '103.8358661'],
 'angsana primary school': ['1.3488946', '103.9516338'],
 'balestier hill primary school': ['1.3256088', '103.8444126'],
 'beacon primary school': ['1.384235', '103.77377'],
 'bedok green primary school': ['1.323989', '103.937761'],
 'bendemeer primary school': ['1.3221999', '103.8650726'],
 'blangah rise primary school': ['1.2761957', '103.8086755'],
 'boon lay garden primary school': ['1.34287', '103.712983'],
 'bukit panjang primary school': ['1.3735964', '103.7695266'],
 'bukit timah primary school': ['1.3

In [209]:
school_lon_lat_df = pd.DataFrame(school_lon_lat, index=['Latitude', 'Longitude']).T
school_lon_lat_df['Flag'] = school_lon_lat_df['Latitude'].apply(lambda x: 0 if x.find('/') == -1 else 1)
school_lon_lat_df[(school_lon_lat_df['Latitude'] == 'attempt') | (school_lon_lat_df['Flag'] == 1)]
school_lon_lat_df.drop(index=school_lon_lat_df[school_lon_lat_df['Flag'] == 1].index, inplace=True)
school_lon_lat_df['Latitude'] = school_lon_lat_df['Latitude'].astype(float)
school_lon_lat_df['Longitude'] = school_lon_lat_df['Longitude'].astype(float)
school_lon_lat_df.drop(columns=['Flag'], inplace=True)

In [177]:
nearest_school_dist = {}

for i in streets_lon_lat.index:
    
    flat_primary_distance = []
    
    for j in school_lon_lat_df.index:
        
        flat_primary_distance.append(distance(lon1=streets_lon_lat.loc[i, 'Longitude'], lat1=streets_lon_lat.loc[i, 'Latitude'], lon2=school_lon_lat_df.loc[j, 'Longitude'], lat2=school_lon_lat_df.loc[j, 'Latitude']))
        
    nearest_school_dist[i] = sorted(list(zip(school_lon_lat_df.index, flat_primary_distance)), key=lambda x: x[1], reverse=False)[:1]

In [210]:
school_lon_lat_df.reset_index(inplace=True)
school_lon_lat_df.rename(columns={'index' : 'School'}, inplace=True)
# school_lon_lat_df.set_index(keys=['School'], inplace=True)
school_lon_lat_df

,School,Latitude,Longitude
0,admiralty primary school,1.443045,103.800309
1,ahmad ibrahim primary school,1.433735,103.832782
2,alexandra primary school,1.291286,103.823573
3,anchor green primary school,1.390925,103.887407
4,anderson primary school,1.384692,103.841225
5,ang mo kio primary school,1.369347,103.839815
6,anglo-chinese school (junior),1.308957,103.841503
7,anglo-chinese school (primary),1.318682,103.835866
8,angsana primary school,1.348895,103.951634
9,balestier hill primary school,1.325609,103.844413


In [191]:
nearest_school_dist

{'ANG MO KIO AVE 1 BLK 309': [('teck ghee primary school', 722.787613952292)],
 'ANG MO KIO AVE 1 BLK 216': [('ang mo kio primary school', 391.835678088125)],
 'ANG MO KIO AVE 3 BLK 211': [('ang mo kio primary school', 198.750995404522)],
 'ANG MO KIO AVE 3 BLK 202': [('ang mo kio primary school',
   539.5449035022103)],
 'ANG MO KIO AVE 3 BLK 235': [('ang mo kio primary school',
   482.40542163679777)],
 'ANG MO KIO AVE 3 BLK 232': [('ang mo kio primary school',
   311.89816405907146)],
 'ANG MO KIO AVE 1 BLK 308': [('teck ghee primary school', 698.0547299035255)],
 'ANG MO KIO AVE 1 BLK 220': [('ang mo kio primary school',
   419.0035376375302)],
 'ANG MO KIO AVE 1 BLK 219': [('ang mo kio primary school',
   380.8405101360845)],
 'ANG MO KIO AVE 3 BLK 247': [('ang mo kio primary school',
   647.3543915437361)],
 'ANG MO KIO AVE 1 BLK 320': [('teck ghee primary school',
   237.93372399991014)],
 'ANG MO KIO AVE 4 BLK 252': [("chij st. nicholas girls' school",
   610.024849820396)],
 '

In [217]:
flat_primary_distance = pd.DataFrame(nearest_school_dist, index=['School']).T
flat_primary_distance['Distance'] = flat_primary_distance['School'].apply(lambda x: x[1])
flat_primary_distance['School'] = flat_primary_distance['School'].apply(lambda x: x[0])
primary_index = flat_primary_distance.index
flat_primary_distance = flat_primary_distance.merge(school_lon_lat_df, how='left', on='School')
flat_primary_distance.index = primary_index
flat_primary_distance

,School,Distance,Latitude,Longitude
ANG MO KIO AVE 1 BLK 309,teck ghee primary school,722.787614,1.365396,103.850524
ANG MO KIO AVE 1 BLK 216,ang mo kio primary school,391.835678,1.369347,103.839815
ANG MO KIO AVE 3 BLK 211,ang mo kio primary school,198.750995,1.369347,103.839815
ANG MO KIO AVE 3 BLK 202,ang mo kio primary school,539.544904,1.369347,103.839815
ANG MO KIO AVE 3 BLK 235,ang mo kio primary school,482.405422,1.369347,103.839815
ANG MO KIO AVE 3 BLK 232,ang mo kio primary school,311.898164,1.369347,103.839815
ANG MO KIO AVE 1 BLK 308,teck ghee primary school,698.054730,1.365396,103.850524
ANG MO KIO AVE 1 BLK 220,ang mo kio primary school,419.003538,1.369347,103.839815
ANG MO KIO AVE 1 BLK 219,ang mo kio primary school,380.840510,1.369347,103.839815
ANG MO KIO AVE 3 BLK 247,ang mo kio primary school,647.354392,1.369347,103.839815


In [218]:
flat_primary_distance.to_csv('./datasets/flat_primary_distance.csv')

## Secondary School

In [219]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://en.wikipedia.org/wiki/List_of_secondary_schools_in_Singapore')

HTML = driver.page_source

driver.close()

In [220]:
school_info = Selector(text=HTML).xpath('//*[@class="wikitable sortable"]/tbody/tr/td/text()').extract()
schools = [col for col in school_info if 'Secondary' in col]
schools += [col for col in Selector(text=HTML).xpath('//*[@class="wikitable sortable"]/tbody/tr/td/a/text()').extract() if 'school' in col.lower()]

for i, school in enumerate(schools):
    schools[i] = school.strip().lower().replace(',','')

schools = np.unique(schools)
schools

array(['admiralty secondary school', 'ahmad ibrahim secondary school',
       'anderson secondary school', 'ang mo kio secondary school',
       'anglican high school', 'anglo-chinese school (barker road)',
       'anglo-chinese school (independent)', 'assumption english school',
       'bartley secondary school', 'beatty secondary school',
       'bedok green secondary school', 'bedok south secondary school',
       'bedok view secondary school', 'bendemeer secondary school',
       'boon lay secondary school', 'bowen secondary school',
       'broadrick secondary school', 'bukit batok secondary school',
       'bukit merah secondary school',
       'bukit panjang government high school',
       'bukit view secondary school', 'canberra secondary school',
       'catholic high school', "cedar girls' secondary school",
       'changkat changi secondary school',
       "chij st. nicholas girls' school",
       'christ church secondary school', 'chua chu kang secondary school',
       'ch

In [224]:
# school_lon_lat = {}
scraping(school_lon_lat_df[(school_lon_lat_df['Latitude'] == 'attempt') | (school_lon_lat_df['Flag'] == 1)].index, school_lon_lat, 'Sec School Loc')

{'admiralty secondary school': ['1.4462673', '103.8026082'],
 'ahmad ibrahim secondary school': ['1.436316', '103.8299037'],
 'anderson secondary school': ['1.3750929', '103.8516001'],
 'ang mo kio secondary school': ['1.367353', '103.8426'],
 'anglican high school': ['1.3307761', '103.9416359'],
 'anglo-chinese school (barker road)': ['1.3189219', '103.8356843'],
 'anglo-chinese school (independent)': ['1.3023151', '103.7809653'],
 'assumption english school': ['1.3684298', '103.766367'],
 'bartley secondary school': ['1.340275', '103.882831'],
 'beatty secondary school': ['1.34154', '103.851956'],
 'bedok green secondary school': ['1.3341416', '103.935336'],
 'bedok south secondary school': ['1.3248089', '103.950977'],
 'bedok view secondary school': ['1.3246274', '103.9438629'],
 'bendemeer secondary school': ['1.3245503', '103.8606227'],
 'boon lay secondary school': ['1.3433488', '103.7018871'],
 'bowen secondary school': ['1.371165', '103.877974'],
 'broadrick secondary school': 

In [226]:
school_lon_lat_df = pd.DataFrame(school_lon_lat, index=['Latitude', 'Longitude']).T
# school_lon_lat_df['Flag'] = school_lon_lat_df['Latitude'].apply(lambda x: 0 if x.find('/') == -1 else 1)
# school_lon_lat_df[(school_lon_lat_df['Latitude'] == 'attempt') | (school_lon_lat_df['Flag'] == 1)]
# school_lon_lat_df.drop(index=school_lon_lat_df[school_lon_lat_df['Flag'] == 1].index, inplace=True)
school_lon_lat_df['Latitude'] = school_lon_lat_df['Latitude'].astype(float)
school_lon_lat_df['Longitude'] = school_lon_lat_df['Longitude'].astype(float)

In [231]:
nearest_school_dist = {}

for i in streets_lon_lat.index:
    
    flat_secondary_distance = []
    
    for j in school_lon_lat_df.index:
        
        flat_secondary_distance.append(distance(lon1=streets_lon_lat.loc[i, 'Longitude'], lat1=streets_lon_lat.loc[i, 'Latitude'], lon2=school_lon_lat_df.loc[j, 'Longitude'], lat2=school_lon_lat_df.loc[j, 'Latitude']))
        
    nearest_school_dist[i] = sorted(list(zip(school_lon_lat_df.index, flat_secondary_distance)), key=lambda x: x[1], reverse=False)[:1]

In [232]:
school_lon_lat_df.reset_index(inplace=True)
school_lon_lat_df.rename(columns={'index' : 'School'}, inplace=True)
school_lon_lat_df

,School,Latitude,Longitude
0,admiralty secondary school,1.446267,103.802608
1,ahmad ibrahim secondary school,1.436316,103.829904
2,anderson secondary school,1.375093,103.851600
3,ang mo kio secondary school,1.367353,103.842600
4,anglican high school,1.330776,103.941636
5,anglo-chinese school (barker road),1.318922,103.835684
6,anglo-chinese school (independent),1.302315,103.780965
7,assumption english school,1.368430,103.766367
8,bartley secondary school,1.340275,103.882831
9,beatty secondary school,1.341540,103.851956


In [239]:
flat_secondary_distance = pd.DataFrame(nearest_school_dist, index=['School']).T
flat_secondary_distance['Distance'] = flat_secondary_distance['School'].apply(lambda x: x[1])
flat_secondary_distance['School'] = flat_secondary_distance['School'].apply(lambda x: x[0])
secondary_index = flat_secondary_distance.index
flat_secondary_distance = flat_secondary_distance.merge(school_lon_lat_df, how='left', on='School')
flat_secondary_distance.index = secondary_index

In [240]:
flat_secondary_distance

,School,Distance,Latitude,Longitude
ANG MO KIO AVE 1 BLK 309,ang mo kio secondary school,375.846945,1.367353,103.842600
ANG MO KIO AVE 1 BLK 216,ang mo kio secondary school,183.087835,1.367353,103.842600
ANG MO KIO AVE 3 BLK 211,ang mo kio secondary school,234.670491,1.367353,103.842600
ANG MO KIO AVE 3 BLK 202,ang mo kio secondary school,252.865519,1.367353,103.842600
ANG MO KIO AVE 3 BLK 235,mayflower secondary school,218.219586,1.368625,103.835212
ANG MO KIO AVE 3 BLK 232,mayflower secondary school,231.200359,1.368625,103.835212
ANG MO KIO AVE 1 BLK 308,ang mo kio secondary school,389.855597,1.367353,103.842600
ANG MO KIO AVE 1 BLK 220,ang mo kio secondary school,309.070514,1.367353,103.842600
ANG MO KIO AVE 1 BLK 219,ang mo kio secondary school,277.119929,1.367353,103.842600
ANG MO KIO AVE 3 BLK 247,mayflower secondary school,137.851200,1.368625,103.835212


In [241]:
flat_secondary_distance.to_csv('./datasets/flat_secondary_distance.csv')

In [ ]:
school_lon_lat_df = pd.read_csv('./')

In [8]:
flat_secondary_distance = []

for j in school_lon_lat_df.index:

    flat_secondary_distance.append(distance(lon1=103.7624299, lat1=1.3660793, lon2=school_lon_lat_df.loc[j, 'Longitude'], lat2=school_lon_lat_df.loc[j, 'Latitude']))

flat_secondary_distance = sorted(list(zip(school_lon_lat_df.index, flat_secondary_distance)), key=lambda x: x[1], reverse=False)[:1]

NameError: name 'school_lon_lat_df' is not defined

In [7]:
flat_station_distance

[('Cashew MRT station', 494.7720741705087),
 ('Hillview MRT station', 649.0064319201534),
 ('Bukit Panjang MRT/LRT station', 1360.607206835853),
 ('Bukit Gombak MRT station', 1415.0023634664055),
 ('Bukit Batok MRT station', 2444.1984967383887)]